                                                                              Tic-Tac -Toe

                                                                        Using  Minimax Algorithm

In [35]:
import json

In [36]:
def game_board(state):
    """
    Displays the Tic-Tac-Toe game board based on the provided state.

    Parameters:state (str): A string representing the current state of the game board.
    Preconditions: The state should be a string of length 9, where each
                    character represents a cell in the Tic-Tac-Toe grid.
                    Valid characters include 'X', 'O', or '-' for an empty cell.
    """
    print('-------------------')
    print('| ', state[0], ' | ', state[1], ' | ', state[2], ' |')
    print('-------------------')
    print('| ', state[3], ' | ', state[4], ' | ', state[5], ' |')
    print('-------------------')
    print('| ', state[6], ' | ', state[7], ' | ', state[8], ' |')
    print('-------------------')

game_board('---------')


-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------


In [37]:
def terminal_state(state):
    """
    Determines the terminal state of a Tic-Tac-Toe game based on the provided state.

    Parameters: state (str): A string representing the current state of the Tic-Tac-Toe game board.
    Preconditions: The state should be a string of length 9, where each
                    character represents a cell in the Tic-Tac-Toe grid.
                    Valid characters include 'X', 'O', or '-' for an empty cell.
    Returns:
        str or False: Returns 'max' if 'X' wins, 'min' if 'O' wins, 'draw' if the game is a draw,
        and False if the game is still ongoing.
    """
    win_combinations = [
        (0, 1, 2), (3, 4, 5), (6, 7, 8),
        (0, 3, 6), (1, 4, 7), (2, 5, 8),
        (0, 4, 8), (2, 4, 6)
    ]

    for pos in win_combinations:
        a, b, c = pos
        if state[a] == state[b] == state[c] == 'X':
            return 'max'
        elif state[a] == state[b] == state[c] == 'O':
            return 'min'

    for cell in state:
        if cell == '-':
            return False  # Game still ongoing

    return 'draw'  # No winner, and the board is full (draw)


In [38]:
def state_maker(initial_state, pos, player):
    """
    Creates a new state by placing the specified player's symbol at the given position.

    Parameters:
    
    initial_state (str): The initial state of the Tic-Tac-Toe game board.
    Preconditions:  Should be a string of length 9, where each character represents
     a cell in the Tic-Tac-Toe grid. Valid characters include 'X','O', or '-' for an empty cell.
     
    pos (int):    The position where the player's symbol will be placed.
    Preconditions: Should be an integer between 0 and 8, inclusive, representing
                    a valid position on the Tic-Tac-Toe grid.

    player (str): The player's symbol ('X' or 'O') to be placed at the specified position.
    Preconditions: Should be a valid player symbol ('X' or 'O').

    Returns:
        str: The new state of the Tic-Tac-Toe game board after the player's move.
    """
    state = ""
    for i, cell in enumerate(initial_state):
        if cell != 'X' and cell != 'O' and i == pos:
            state += player
        else:
            state += cell

    return state

print(state_maker('O--------', 5, 'X'))


O----X---


In [39]:
def tic_tac_toe_tree(state, player):
    """
    Generates a game tree for the Tic-Tac-Toe game based on the current state and player.

    Parameters:
    
    state (str): The current state of the Tic-Tac-Toe game board.
    Preconditions: Should be a string of length 9, where each character represents
    a cell in the Tic-Tac-Toe grid. Valid characters include 'X','O', or '-' for an empty cell.

    player (str): The current player's symbol ('X' or 'O') whose move is being considered.
    Preconditions: Should be a valid player symbol ('X' or 'O').

    Returns:
        dict: A dictionary representing the game tree with information about the current state,
        the player, the score, and the possible children states.
    """
    val = terminal_state(state)
    if val:
        return {
            'current_state': state,
            'player': player,
            'score': 1 if val == 'max' else -1 if val == 'min' else 0,
            'children': []
        }

    tree = {
        'current_state': state,
        'player': player,
        'score': 0,
        'children': []
    }

    for i, cell in enumerate(state):
        if cell == '-':
            state1 = state_maker(state, i, player)
            child_tree = tic_tac_toe_tree(state1, 'O' if player == 'X' else 'X')
            tree['children'].append(child_tree)

    if player == 'X':
        tree['score'] = max(child['score'] for child in tree['children'])

    if player == 'O':
        tree['score'] = min(child['score'] for child in tree['children'])

    return tree

initial_state = '---------'
game_tree = tic_tac_toe_tree(initial_state, 'O')


In [40]:
graph = {}  #graph is the global variable holding the graph data

def tree_to_graph(dic):
    """
    Converts a Tic-Tac-Toe game tree represented by a dictionary into a graph.

    Parameters:
    
        dic (dict): A dictionary representing a node in the Tic-Tac-Toe game tree.
        Preconditions: Should have keys 'current_state', 'score', and 'children',
        where 'current_state' is the state of the game board,'score' is the score
        associated with the node, and 'children' is a list of child nodes.
    """
    state = dic['current_state']
    score = dic['score']

    if state not in graph:
        graph[state] = score

    for child in dic['children']:
        tree_to_graph(child)

    return

tree_to_graph(game_tree)


In [41]:

with open('minimax_json.json', 'w') as file:
    """
    Writes the Tic-Tac-Toe game tree graph to a JSON file.
    Parameters: 
      f: File object representing the opened file for writing.
    """
    json.dump(graph, file)


In [42]:
def ai_turn(player_state):
    """
    Determines the optimal move for the AI player ('X') based on the current player_state.

    Parameters:
    player_state (str): The current state of the Tic-Tac-Toe game board for the player.
    Preconditions: Should be a string of length 9, where each character represents
    a cell in the Tic-Tac-Toe grid. Valid characters include 'X','O', or '-' for an empty cell.

    Returns:
        str: The optimal next state for the AI player.
    """
    possible_state_lst = []

    # Generate possible next states
    for i, cell in enumerate(player_state):
        if cell == "-":
            next_state = state_maker(player_state, i, 'X')
            possible_state_lst.append(next_state)

    best_value = float("inf")
    best_move = player_state

    for possible_state in possible_state_lst:
        if possible_state in graph:
            # Check if the score of the possible state matches the score of the current state
            if graph[possible_state] == graph[player_state]:
                return possible_state


In [43]:
next_state = '---------'

lst = []

while True:
    game_board(next_state)
    index = int(input('Enter the the cell number from 0 to 8: '))

    next_state = state_maker(next_state, index, 'O')
    val = terminal_state(next_state)
    if val:
        if val == 'min':
            print('You Won')

        elif val == 'max':
            print('AI Won')
        else:
            print('Game Draw!')

        break

    next_state = ai_turn(next_state)
    val = terminal_state(next_state)
    if val:
        game_board(next_state)
        if val == 'min':
            print('You Won')

        elif val == 'max':
            print('AI Won')
        else:
            print('Game Draw!')
            
        break


-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
-------------------
|  O  |  -  |  -  |
-------------------
|  -  |  X  |  -  |
-------------------
|  -  |  -  |  -  |
-------------------
-------------------
|  O  |  -  |  -  |
-------------------
|  X  |  X  |  -  |
-------------------
|  O  |  -  |  -  |
-------------------
-------------------
|  O  |  X  |  -  |
-------------------
|  X  |  X  |  O  |
-------------------
|  O  |  -  |  -  |
-------------------
-------------------
|  O  |  X  |  -  |
-------------------
|  X  |  X  |  O  |
-------------------
|  O  |  O  |  X  |
-------------------
Game Draw!
